In [12]:
import pandas as pd
import requests
import os
from urllib.parse import urlparse
from time import sleep
import csv

In [ ]:
BRAVE_API_KEY = os.getenv("BRAVE_API_KEY")
MAX_QUERIES = os.getenv("MAX_QUERY_COUNT")
query_count = 0

In [14]:
import random
def extract_links(city, state):
    global query_count
    result = {
        "city": city,
        "state": state,
        "official_site": "NOT_FOUND",
        "utility_sites": []
    }
    if query_count >= MAX_QUERIES:
        print("Reached Brave API limit. Stopping...")
        raise StopIteration

    # --- OFFICIAL SITE ---
    official_query = f"{city} {state} official city site"
    official_links = brave_search(official_query)
    sleep(random.uniform(1.2, 1.6))

    if official_links:
        result["official_site"] = official_links[0]

    for link in official_links:
        if "wikipedia.org" in link.lower():
            continue 
        if "facebook.com" in link.lower():
            continue 
        result["official_site"] = link
        break
    if query_count >= MAX_QUERIES:
        print("Reached Brave API limit. Stopping...")
        raise StopIteration
    
    print("query count: ", query_count)

    # --- UTILITY SITES ---
    utility_query = f"{city} {state} pay utility water sewer electric bill site"
    utility_links = brave_search(utility_query, num=15)
    sleep(random.uniform(1.2, 1.6))

    if utility_links:
        seen_domains = set()
        cleaned = []
        for link in utility_links:
            domain = urlparse(link).netloc
            if domain not in seen_domains:
                cleaned.append(link)
                seen_domains.add(domain)
            if len(cleaned) == 5:
                break
        result["utility_sites"] = cleaned
    else:
        result["utility_sites"] = ["NOT_FOUND"]
    return result


def brave_search(query, num=10):
    url = "https://api.search.brave.com/res/v1/web/search"
    headers = {
        "Accept": "application/json",
        "X-Subscription-Token": BRAVE_API_KEY
    }
    params = {
        "q": query,
        "count": num
    }

    try:
        res = requests.get(url, headers=headers, params=params, timeout=10)
        res.raise_for_status()
        data = res.json()

        web_results = data.get("web", {}).get("results", [])
        if not web_results:
            print(f"No results for: {query}")
            return []

        return [item.get("url", "") for item in web_results if "url" in item]

    except requests.exceptions.HTTPError as e:
        print(f"Brave API HTTP error for '{query}': {e}")
        return []
    except Exception as e:
        print(f"Brave API general error for '{query}': {e}")
        return []


In [15]:
df = pd.read_csv("us_cities.csv")

output_file = "verified_city_utilities.csv"
verified_set = set()
if os.path.exists(output_file):
    try:
        verified_df = pd.read_csv(output_file, usecols=[0, 1], header=0, on_bad_lines="skip")
        verified_set = set(zip(
            verified_df["city"].str.lower().str.strip(),
            verified_df["state"].str.lower().str.strip()
        ))
    except Exception as e:
        print(f"⚠️ Error loading verified file: {e}")
        verified_set = set()

print(f"{len(verified_set)} cities already verified. Skipping those.")
write_header = not os.path.exists(output_file) or os.stat(output_file).st_size == 0

with open(output_file, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["city", "state", "official_site", "utility_sites"],
        quoting=csv.QUOTE_MINIMAL
    )

    if write_header:
        writer.writeheader()

    for _, row in df.iterrows():
        city = row["CITY"].strip()
        state = row["STATE_NAME"].strip()
        key = (city.lower(), state.lower())

        if key in verified_set:
            print(f"⏭️ Skipping already verified: {city}, {state}")
            continue

        print(f"Searching: {city}, {state}")
        try:
            info = extract_links(city, state)
            query_count += 2
            writer.writerow({
                "city": info["city"],
                "state": info["state"],
                "official_site": info["official_site"],
                "utility_sites": "; ".join(info["utility_sites"])
            })
            f.flush()
        except Exception as e:
            print(f"Error with {city}, {state}: {e}")


FileNotFoundError: [Errno 2] No such file or directory: 'us_cities.csv'